In [1]:
import configparser
import csv
import os
import os.path as ops
import pickle

from PIL import Image
import numpy as np
import scipy
import torch

import matplotlib.pyplot as plt
import argparse
import glob
import json
import cv2
import sys
sys.path.append("..")
from dataset.cosmos_dataset import COSMOSDatasetPedestrian, COSMOSDatasetPedestrian_AUG, COSMOSDatasetPedestrianTtoD

In [2]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# Get Faster R-CNN

In [3]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
def get_detection_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.roi_heads.nms_thresh = 0.3
    
    return model

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
def get_detection_model2(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    size = ((16,), (32,), (64,), (128,), (256,), (512,))
    aspect_ratios = ((0.5, 1.0, 2.0),
                     (0.5, 1.0, 2.0),
                     (0.5, 1.0, 2.0),
                     (0.5, 1.0, 2.0),
                     (0.5, 1.0, 2.0),
                     (0.5, 1.0, 2.0))
    model.rpn.anchor_generator = AnchorGenerator(size, aspect_ratios)
    
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.roi_heads.nms_thresh = 0.3
    
    return model

# Define the dataset

In [5]:
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [6]:
class COSMOSDatasetPedestrianOandA(torch.utils.data.Dataset):
    def __init__(self, dataset_org, dataset_aug):
        self.dataset_org = dataset_org
        self.dataset_aug = dataset_aug
        self.dataset_org_len = len(dataset_org)
        self.dataset_aug_len = len(dataset_aug)
        self._classes = ('Background', 'Pedestrian')
    @property
    def num_classes(self):
        return len(self._classes)
    def __getitem__(self, idx):
        if idx < self.dataset_org_len:
            return self.dataset_org[idx]
        else:
            return self.dataset_aug[idx-self.dataset_org_len]
    def __len__(self):
        return self.dataset_org_len+self.dataset_aug_len

In [7]:
dataset_path = '../../COSMOS_data_tracktor/detection_dataset/maskrcnn_lablled-bbox-train-val'
dataset_path_aug = '../../COSMOS_data_tracktor/detection_dataset/detection_dataset3'

dataset_org = COSMOSDatasetPedestrian(dataset_path, get_transform(train=False))
dataset_aug = COSMOSDatasetPedestrian_AUG(dataset_path_aug, get_transform(train=False))

dataset = COSMOSDatasetPedestrianOandA(dataset_org, dataset_aug)

torch.manual_seed(1)
data_loader_train = torch.utils.data.DataLoader(dataset, batch_size=2, 
                                          shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

In [8]:
print(len(dataset_org))
print(len(dataset_aug))
print(len(dataset))

1506
1380
2886


# Training

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# get the model using our helper function
model = get_detection_model2(dataset.num_classes)

#model_path = '/home/ecbm4040/COSMOS_data/frcnn_cosmos_output/faster_rcnn_fpn_training_cosmos/model_vehicle_O&A_second_epoch_20.model'

#model.load_state_dict(torch.load(model_path))


# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
# optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [9]:
def evaluate_and_write_result_files(model, parent_set, data_loader, output_dir):
    model.eval()
    results = {}
    for imgs, targets in data_loader:
        imgs = [img.to(device) for img in imgs]
        with torch.no_grad():
            preds = model(imgs)
        for pred, target in zip(preds, targets):
            results[target['image_id'].item()] = {'boxes': pred['boxes'].cpu(),
                                                  'labels': pred['labels'].cpu(),
                                                  'scores': pred['scores'].cpu()}
    dataset_len = len(data_loader.dataset)
    parent_set.print_eval(dataset_len, results, ovthresh=0.5)
    # parent_set.print_eval(dataset_len, results, ovthresh=0.75)
    #parent_set.write_results_files(results, output_dir) 

In [10]:
output_dir = None
dataset_path_tracking = '../../COSMOS_data_tracktor/Track_data_cosmos/only_p/traffic_video_GP020614_190720_0237_0407_90sec_calibrated'
dataset_test3 = COSMOSDatasetPedestrianTtoD(dataset_path_tracking, get_transform(train=False))
torch.manual_seed(1)
data_loader_test3 = torch.utils.data.DataLoader(dataset_test3, batch_size=1, 
                                                    shuffle=False, num_workers=4, collate_fn=utils.collate_fn)
evaluate_and_write_result_files(model, dataset_test3, data_loader_test3, output_dir)

In [12]:
num_epochs = 40
output_dir = None

for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=40)
    lr_scheduler.step()
    if epoch % 4 == 0:
        # evaluate_and_write_result_files(model, dataset, data_loader_valid, output_dir)
        evaluate_and_write_result_files(model, dataset_test3, data_loader_test3, output_dir)
        torch.save(model.state_dict(), 
                       f"../../COSMOS_data_tracktor/frcnn_cosmos_output/faster_rcnn_fpn_training_cosmos_P/model_pedestrian_O&A_second_adam_epoch_{epoch}.model")

Epoch: [1]  [   0/1443]  eta: 1:25:49  lr: 0.001000  loss: 1.6195 (1.6195)  loss_classifier: 1.2656 (1.2656)  loss_box_reg: 0.0066 (0.0066)  loss_objectness: 0.3209 (0.3209)  loss_rpn_box_reg: 0.0263 (0.0263)  time: 3.5689  data: 0.7799  max mem: 2509
Epoch: [1]  [  40/1443]  eta: 0:10:36  lr: 0.001000  loss: 0.3580 (0.4101)  loss_classifier: 0.1870 (0.2091)  loss_box_reg: 0.0816 (0.0719)  loss_objectness: 0.0427 (0.0849)  loss_rpn_box_reg: 0.0214 (0.0442)  time: 0.3748  data: 0.0131  max mem: 2775
Epoch: [1]  [  80/1443]  eta: 0:09:23  lr: 0.001000  loss: 0.2583 (0.3651)  loss_classifier: 0.1102 (0.1708)  loss_box_reg: 0.1065 (0.1015)  loss_objectness: 0.0191 (0.0558)  loss_rpn_box_reg: 0.0142 (0.0371)  time: 0.3721  data: 0.0122  max mem: 2775
Epoch: [1]  [ 120/1443]  eta: 0:08:50  lr: 0.001000  loss: 0.2712 (0.3380)  loss_classifier: 0.0982 (0.1496)  loss_box_reg: 0.1113 (0.1100)  loss_objectness: 0.0134 (0.0432)  loss_rpn_box_reg: 0.0145 (0.0352)  time: 0.3739  data: 0.0139  max me

Epoch: [1]  [1320/1443]  eta: 0:00:46  lr: 0.001000  loss: 0.2643 (0.2660)  loss_classifier: 0.0996 (0.1116)  loss_box_reg: 0.1356 (0.1183)  loss_objectness: 0.0044 (0.0126)  loss_rpn_box_reg: 0.0132 (0.0234)  time: 0.3730  data: 0.0127  max mem: 2775
Epoch: [1]  [1360/1443]  eta: 0:00:31  lr: 0.001000  loss: 0.2087 (0.2656)  loss_classifier: 0.0938 (0.1113)  loss_box_reg: 0.1012 (0.1179)  loss_objectness: 0.0038 (0.0126)  loss_rpn_box_reg: 0.0092 (0.0238)  time: 0.3735  data: 0.0132  max mem: 2775
Epoch: [1]  [1400/1443]  eta: 0:00:16  lr: 0.001000  loss: 0.2046 (0.2642)  loss_classifier: 0.0799 (0.1108)  loss_box_reg: 0.0911 (0.1174)  loss_objectness: 0.0044 (0.0124)  loss_rpn_box_reg: 0.0110 (0.0236)  time: 0.3719  data: 0.0124  max mem: 2775
Epoch: [1]  [1440/1443]  eta: 0:00:01  lr: 0.001000  loss: 0.2381 (0.2636)  loss_classifier: 0.0972 (0.1105)  loss_box_reg: 0.0887 (0.1171)  loss_objectness: 0.0040 (0.0123)  loss_rpn_box_reg: 0.0102 (0.0237)  time: 0.3709  data: 0.0132  max me

Epoch: [2]  [1120/1443]  eta: 0:02:01  lr: 0.001000  loss: 0.2003 (0.2247)  loss_classifier: 0.0876 (0.0943)  loss_box_reg: 0.0978 (0.1058)  loss_objectness: 0.0021 (0.0052)  loss_rpn_box_reg: 0.0122 (0.0193)  time: 0.3730  data: 0.0121  max mem: 2775
Epoch: [2]  [1160/1443]  eta: 0:01:46  lr: 0.001000  loss: 0.2203 (0.2255)  loss_classifier: 0.0893 (0.0945)  loss_box_reg: 0.1136 (0.1065)  loss_objectness: 0.0016 (0.0052)  loss_rpn_box_reg: 0.0113 (0.0193)  time: 0.3731  data: 0.0116  max mem: 2775
Epoch: [2]  [1200/1443]  eta: 0:01:31  lr: 0.001000  loss: 0.1906 (0.2256)  loss_classifier: 0.0803 (0.0946)  loss_box_reg: 0.0883 (0.1067)  loss_objectness: 0.0008 (0.0052)  loss_rpn_box_reg: 0.0096 (0.0192)  time: 0.3744  data: 0.0124  max mem: 2775
Epoch: [2]  [1240/1443]  eta: 0:01:16  lr: 0.001000  loss: 0.1910 (0.2251)  loss_classifier: 0.0819 (0.0943)  loss_box_reg: 0.0909 (0.1065)  loss_objectness: 0.0032 (0.0051)  loss_rpn_box_reg: 0.0076 (0.0192)  time: 0.3744  data: 0.0115  max me

Epoch: [3]  [ 920/1443]  eta: 0:03:17  lr: 0.001000  loss: 0.1591 (0.2017)  loss_classifier: 0.0726 (0.0829)  loss_box_reg: 0.0777 (0.0976)  loss_objectness: 0.0015 (0.0039)  loss_rpn_box_reg: 0.0099 (0.0173)  time: 0.3764  data: 0.0111  max mem: 2775
Epoch: [3]  [ 960/1443]  eta: 0:03:02  lr: 0.001000  loss: 0.2508 (0.2028)  loss_classifier: 0.0870 (0.0834)  loss_box_reg: 0.1171 (0.0981)  loss_objectness: 0.0026 (0.0039)  loss_rpn_box_reg: 0.0163 (0.0174)  time: 0.3770  data: 0.0110  max mem: 2775
Epoch: [3]  [1000/1443]  eta: 0:02:47  lr: 0.001000  loss: 0.1460 (0.2025)  loss_classifier: 0.0727 (0.0834)  loss_box_reg: 0.0734 (0.0980)  loss_objectness: 0.0029 (0.0039)  loss_rpn_box_reg: 0.0075 (0.0173)  time: 0.3764  data: 0.0114  max mem: 2775
Epoch: [3]  [1040/1443]  eta: 0:02:32  lr: 0.001000  loss: 0.1921 (0.2019)  loss_classifier: 0.0832 (0.0833)  loss_box_reg: 0.0927 (0.0976)  loss_objectness: 0.0012 (0.0039)  loss_rpn_box_reg: 0.0114 (0.0172)  time: 0.3783  data: 0.0121  max me

Epoch: [4]  [ 720/1443]  eta: 0:04:33  lr: 0.001000  loss: 0.1905 (0.1885)  loss_classifier: 0.0758 (0.0767)  loss_box_reg: 0.0991 (0.0924)  loss_objectness: 0.0009 (0.0032)  loss_rpn_box_reg: 0.0078 (0.0163)  time: 0.3765  data: 0.0110  max mem: 2775
Epoch: [4]  [ 760/1443]  eta: 0:04:18  lr: 0.001000  loss: 0.1726 (0.1879)  loss_classifier: 0.0670 (0.0765)  loss_box_reg: 0.0895 (0.0921)  loss_objectness: 0.0013 (0.0031)  loss_rpn_box_reg: 0.0078 (0.0161)  time: 0.3775  data: 0.0113  max mem: 2775
Epoch: [4]  [ 800/1443]  eta: 0:04:02  lr: 0.001000  loss: 0.2063 (0.1892)  loss_classifier: 0.0870 (0.0771)  loss_box_reg: 0.0999 (0.0928)  loss_objectness: 0.0007 (0.0031)  loss_rpn_box_reg: 0.0092 (0.0162)  time: 0.3774  data: 0.0113  max mem: 2775
Epoch: [4]  [ 840/1443]  eta: 0:03:47  lr: 0.001000  loss: 0.1694 (0.1892)  loss_classifier: 0.0661 (0.0772)  loss_box_reg: 0.0820 (0.0928)  loss_objectness: 0.0005 (0.0032)  loss_rpn_box_reg: 0.0074 (0.0161)  time: 0.3761  data: 0.0112  max me

...

Epoch: [37]  [ 360/1443]  eta: 0:06:50  lr: 0.000010  loss: 0.0644 (0.0749)  loss_classifier: 0.0276 (0.0342)  loss_box_reg: 0.0319 (0.0298)  loss_objectness: 0.0002 (0.0010)  loss_rpn_box_reg: 0.0041 (0.0098)  time: 0.3770  data: 0.0120  max mem: 2775
Epoch: [37]  [ 400/1443]  eta: 0:06:35  lr: 0.000010  loss: 0.0766 (0.0748)  loss_classifier: 0.0322 (0.0342)  loss_box_reg: 0.0314 (0.0297)  loss_objectness: 0.0004 (0.0011)  loss_rpn_box_reg: 0.0058 (0.0099)  time: 0.3764  data: 0.0117  max mem: 2775
Epoch: [37]  [ 440/1443]  eta: 0:06:19  lr: 0.000010  loss: 0.0557 (0.0747)  loss_classifier: 0.0290 (0.0342)  loss_box_reg: 0.0195 (0.0295)  loss_objectness: 0.0002 (0.0011)  loss_rpn_box_reg: 0.0040 (0.0099)  time: 0.3766  data: 0.0118  max mem: 2775
Epoch: [37]  [ 480/1443]  eta: 0:06:04  lr: 0.000010  loss: 0.0698 (0.0754)  loss_classifier: 0.0329 (0.0344)  loss_box_reg: 0.0361 (0.0297)  loss_objectness: 0.0004 (0.0011)  loss_rpn_box_reg: 0.0053 (0.0102)  time: 0.3780  data: 0.0117  ma

Epoch: [38]  [ 160/1443]  eta: 0:08:09  lr: 0.000010  loss: 0.0749 (0.0755)  loss_classifier: 0.0270 (0.0343)  loss_box_reg: 0.0289 (0.0286)  loss_objectness: 0.0003 (0.0010)  loss_rpn_box_reg: 0.0048 (0.0116)  time: 0.3757  data: 0.0116  max mem: 2775
Epoch: [38]  [ 200/1443]  eta: 0:07:53  lr: 0.000010  loss: 0.0660 (0.0749)  loss_classifier: 0.0286 (0.0340)  loss_box_reg: 0.0308 (0.0289)  loss_objectness: 0.0002 (0.0010)  loss_rpn_box_reg: 0.0040 (0.0110)  time: 0.3772  data: 0.0116  max mem: 2775
Epoch: [38]  [ 240/1443]  eta: 0:07:37  lr: 0.000010  loss: 0.0703 (0.0737)  loss_classifier: 0.0383 (0.0337)  loss_box_reg: 0.0278 (0.0288)  loss_objectness: 0.0002 (0.0010)  loss_rpn_box_reg: 0.0048 (0.0103)  time: 0.3771  data: 0.0118  max mem: 2775
Epoch: [38]  [ 280/1443]  eta: 0:07:21  lr: 0.000010  loss: 0.0637 (0.0739)  loss_classifier: 0.0331 (0.0338)  loss_box_reg: 0.0268 (0.0286)  loss_objectness: 0.0002 (0.0010)  loss_rpn_box_reg: 0.0031 (0.0104)  time: 0.3769  data: 0.0114  ma

Epoch: [38]  [1442/1443]  eta: 0:00:00  lr: 0.000010  loss: 0.0840 (0.0762)  loss_classifier: 0.0367 (0.0348)  loss_box_reg: 0.0320 (0.0301)  loss_objectness: 0.0003 (0.0010)  loss_rpn_box_reg: 0.0045 (0.0103)  time: 0.3708  data: 0.0114  max mem: 2775
Epoch: [38] Total time: 0:09:04 (0.3770 s / it)
Epoch: [39]  [   0/1443]  eta: 0:23:43  lr: 0.000010  loss: 0.1585 (0.1585)  loss_classifier: 0.0887 (0.0887)  loss_box_reg: 0.0556 (0.0556)  loss_objectness: 0.0070 (0.0070)  loss_rpn_box_reg: 0.0074 (0.0074)  time: 0.9863  data: 0.4986  max mem: 2775
Epoch: [39]  [  40/1443]  eta: 0:09:13  lr: 0.000010  loss: 0.0698 (0.0743)  loss_classifier: 0.0299 (0.0351)  loss_box_reg: 0.0297 (0.0300)  loss_objectness: 0.0001 (0.0011)  loss_rpn_box_reg: 0.0053 (0.0082)  time: 0.3762  data: 0.0115  max mem: 2775
Epoch: [39]  [  80/1443]  eta: 0:08:45  lr: 0.000010  loss: 0.0653 (0.0752)  loss_classifier: 0.0324 (0.0349)  loss_box_reg: 0.0270 (0.0296)  loss_objectness: 0.0001 (0.0010)  loss_rpn_box_reg:

Epoch: [39]  [1280/1443]  eta: 0:01:01  lr: 0.000010  loss: 0.0655 (0.0770)  loss_classifier: 0.0332 (0.0354)  loss_box_reg: 0.0286 (0.0300)  loss_objectness: 0.0002 (0.0011)  loss_rpn_box_reg: 0.0038 (0.0105)  time: 0.3765  data: 0.0115  max mem: 2775
Epoch: [39]  [1320/1443]  eta: 0:00:46  lr: 0.000010  loss: 0.0641 (0.0765)  loss_classifier: 0.0327 (0.0352)  loss_box_reg: 0.0260 (0.0299)  loss_objectness: 0.0001 (0.0010)  loss_rpn_box_reg: 0.0043 (0.0103)  time: 0.3772  data: 0.0120  max mem: 2775
Epoch: [39]  [1360/1443]  eta: 0:00:31  lr: 0.000010  loss: 0.0683 (0.0767)  loss_classifier: 0.0353 (0.0353)  loss_box_reg: 0.0296 (0.0300)  loss_objectness: 0.0003 (0.0010)  loss_rpn_box_reg: 0.0046 (0.0104)  time: 0.3778  data: 0.0118  max mem: 2775
Epoch: [39]  [1400/1443]  eta: 0:00:16  lr: 0.000010  loss: 0.0749 (0.0767)  loss_classifier: 0.0328 (0.0352)  loss_box_reg: 0.0301 (0.0300)  loss_objectness: 0.0003 (0.0010)  loss_rpn_box_reg: 0.0060 (0.0104)  time: 0.3767  data: 0.0114  ma

Epoch: [40]  [1080/1443]  eta: 0:02:17  lr: 0.000010  loss: 0.0505 (0.0751)  loss_classifier: 0.0217 (0.0344)  loss_box_reg: 0.0203 (0.0297)  loss_objectness: 0.0001 (0.0010)  loss_rpn_box_reg: 0.0037 (0.0099)  time: 0.3761  data: 0.0118  max mem: 2775
Epoch: [40]  [1120/1443]  eta: 0:02:01  lr: 0.000010  loss: 0.0711 (0.0752)  loss_classifier: 0.0309 (0.0345)  loss_box_reg: 0.0340 (0.0298)  loss_objectness: 0.0003 (0.0010)  loss_rpn_box_reg: 0.0054 (0.0099)  time: 0.3765  data: 0.0118  max mem: 2775
Epoch: [40]  [1160/1443]  eta: 0:01:46  lr: 0.000010  loss: 0.0714 (0.0758)  loss_classifier: 0.0315 (0.0347)  loss_box_reg: 0.0313 (0.0300)  loss_objectness: 0.0002 (0.0010)  loss_rpn_box_reg: 0.0037 (0.0101)  time: 0.3764  data: 0.0116  max mem: 2775
Epoch: [40]  [1200/1443]  eta: 0:01:31  lr: 0.000010  loss: 0.0394 (0.0757)  loss_classifier: 0.0220 (0.0347)  loss_box_reg: 0.0144 (0.0299)  loss_objectness: 0.0001 (0.0010)  loss_rpn_box_reg: 0.0032 (0.0101)  time: 0.3778  data: 0.0123  ma

# Train another model

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# get the model using our helper function
model = get_detection_model(dataset.num_classes)

#model_path = '/home/ecbm4040/COSMOS_data/frcnn_cosmos_output/faster_rcnn_fpn_training_cosmos/model_vehicle_O&A_second_epoch_20.model'

#model.load_state_dict(torch.load(model_path))


# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
# optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [14]:
def evaluate_and_write_result_files(model, parent_set, data_loader, output_dir):
    model.eval()
    results = {}
    for imgs, targets in data_loader:
        imgs = [img.to(device) for img in imgs]
        with torch.no_grad():
            preds = model(imgs)
        for pred, target in zip(preds, targets):
            results[target['image_id'].item()] = {'boxes': pred['boxes'].cpu(),
                                                  'labels': pred['labels'].cpu(),
                                                  'scores': pred['scores'].cpu()}
    dataset_len = len(data_loader.dataset)
    parent_set.print_eval(dataset_len, results, ovthresh=0.5)
    # parent_set.print_eval(dataset_len, results, ovthresh=0.75)
    #parent_set.write_results_files(results, output_dir) 

In [15]:
output_dir = None
dataset_path_tracking = '../../COSMOS_data_tracktor/Track_data_cosmos/only_p/traffic_video_GP020614_190720_0237_0407_90sec_calibrated'
dataset_test3 = COSMOSDatasetPedestrianTtoD(dataset_path_tracking, get_transform(train=False))
torch.manual_seed(1)
data_loader_test3 = torch.utils.data.DataLoader(dataset_test3, batch_size=1, 
                                                    shuffle=False, num_workers=4, collate_fn=utils.collate_fn)
evaluate_and_write_result_files(model, dataset_test3, data_loader_test3, output_dir)

AP: 0.0004580370530152388 Prec: 0.005494994438264738 Rec: 0.07547746371275783 TP: 494.0 FP: 89406.0


In [16]:
num_epochs = 40
output_dir = None

for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=40)
    lr_scheduler.step()
    if epoch % 4 == 0:
        # evaluate_and_write_result_files(model, dataset, data_loader_valid, output_dir)
        evaluate_and_write_result_files(model, dataset_test3, data_loader_test3, output_dir)
        torch.save(model.state_dict(), 
                       f"../../COSMOS_data_tracktor/frcnn_cosmos_output/faster_rcnn_fpn_training_cosmos_P/model_pedestrian_O&A_second_nochange_adam_epoch_{epoch}.model")

Epoch: [1]  [   0/1443]  eta: 0:25:05  lr: 0.001000  loss: 5.6919 (5.6919)  loss_classifier: 0.9736 (0.9736)  loss_box_reg: 0.0059 (0.0059)  loss_objectness: 4.0690 (4.0690)  loss_rpn_box_reg: 0.6434 (0.6434)  time: 1.0433  data: 0.5975  max mem: 2775
Epoch: [1]  [  40/1443]  eta: 0:09:09  lr: 0.001000  loss: 0.5081 (0.7498)  loss_classifier: 0.1350 (0.2077)  loss_box_reg: 0.1136 (0.1098)  loss_objectness: 0.0647 (0.2094)  loss_rpn_box_reg: 0.2024 (0.2229)  time: 0.3732  data: 0.0127  max mem: 2775
Epoch: [1]  [  80/1443]  eta: 0:08:43  lr: 0.001000  loss: 0.3688 (0.6196)  loss_classifier: 0.1051 (0.1644)  loss_box_reg: 0.1309 (0.1240)  loss_objectness: 0.0262 (0.1302)  loss_rpn_box_reg: 0.1195 (0.2011)  time: 0.3736  data: 0.0122  max mem: 2775
Epoch: [1]  [ 120/1443]  eta: 0:08:23  lr: 0.001000  loss: 0.4292 (0.5603)  loss_classifier: 0.0937 (0.1439)  loss_box_reg: 0.1372 (0.1284)  loss_objectness: 0.0293 (0.0983)  loss_rpn_box_reg: 0.1412 (0.1897)  time: 0.3731  data: 0.0128  max me

Epoch: [1]  [1320/1443]  eta: 0:00:46  lr: 0.001000  loss: 0.3652 (0.3882)  loss_classifier: 0.0831 (0.0927)  loss_box_reg: 0.1182 (0.1137)  loss_objectness: 0.0165 (0.0306)  loss_rpn_box_reg: 0.1287 (0.1512)  time: 0.3810  data: 0.0165  max mem: 2775
Epoch: [1]  [1360/1443]  eta: 0:00:31  lr: 0.001000  loss: 0.3217 (0.3874)  loss_classifier: 0.0709 (0.0923)  loss_box_reg: 0.0911 (0.1131)  loss_objectness: 0.0125 (0.0303)  loss_rpn_box_reg: 0.1291 (0.1516)  time: 0.3762  data: 0.0145  max mem: 2775
Epoch: [1]  [1400/1443]  eta: 0:00:16  lr: 0.001000  loss: 0.2888 (0.3849)  loss_classifier: 0.0719 (0.0918)  loss_box_reg: 0.0842 (0.1124)  loss_objectness: 0.0139 (0.0299)  loss_rpn_box_reg: 0.1380 (0.1509)  time: 0.3768  data: 0.0158  max mem: 2775
Epoch: [1]  [1440/1443]  eta: 0:00:01  lr: 0.001000  loss: 0.2995 (0.3844)  loss_classifier: 0.0730 (0.0915)  loss_box_reg: 0.0764 (0.1121)  loss_objectness: 0.0147 (0.0295)  loss_rpn_box_reg: 0.1274 (0.1513)  time: 0.3716  data: 0.0137  max me

Epoch: [2]  [1120/1443]  eta: 0:02:01  lr: 0.001000  loss: 0.2112 (0.3083)  loss_classifier: 0.0560 (0.0711)  loss_box_reg: 0.0611 (0.0901)  loss_objectness: 0.0066 (0.0155)  loss_rpn_box_reg: 0.1022 (0.1316)  time: 0.3750  data: 0.0112  max mem: 2775
Epoch: [2]  [1160/1443]  eta: 0:01:46  lr: 0.001000  loss: 0.2719 (0.3080)  loss_classifier: 0.0602 (0.0711)  loss_box_reg: 0.0726 (0.0902)  loss_objectness: 0.0087 (0.0155)  loss_rpn_box_reg: 0.1073 (0.1313)  time: 0.3773  data: 0.0118  max mem: 2775
Epoch: [2]  [1200/1443]  eta: 0:01:31  lr: 0.001000  loss: 0.2401 (0.3084)  loss_classifier: 0.0586 (0.0710)  loss_box_reg: 0.0673 (0.0902)  loss_objectness: 0.0102 (0.0156)  loss_rpn_box_reg: 0.1022 (0.1317)  time: 0.3771  data: 0.0116  max mem: 2775
Epoch: [2]  [1240/1443]  eta: 0:01:16  lr: 0.001000  loss: 0.2850 (0.3083)  loss_classifier: 0.0696 (0.0711)  loss_box_reg: 0.0819 (0.0901)  loss_objectness: 0.0075 (0.0156)  loss_rpn_box_reg: 0.1207 (0.1315)  time: 0.3766  data: 0.0118  max me

Epoch: [3]  [ 920/1443]  eta: 0:03:17  lr: 0.001000  loss: 0.2664 (0.2838)  loss_classifier: 0.0664 (0.0650)  loss_box_reg: 0.0863 (0.0840)  loss_objectness: 0.0088 (0.0131)  loss_rpn_box_reg: 0.1161 (0.1217)  time: 0.3768  data: 0.0124  max mem: 2775
Epoch: [3]  [ 960/1443]  eta: 0:03:02  lr: 0.001000  loss: 0.2393 (0.2835)  loss_classifier: 0.0590 (0.0648)  loss_box_reg: 0.0667 (0.0837)  loss_objectness: 0.0129 (0.0131)  loss_rpn_box_reg: 0.1170 (0.1219)  time: 0.3757  data: 0.0119  max mem: 2775
Epoch: [3]  [1000/1443]  eta: 0:02:46  lr: 0.001000  loss: 0.2315 (0.2814)  loss_classifier: 0.0511 (0.0643)  loss_box_reg: 0.0586 (0.0829)  loss_objectness: 0.0089 (0.0130)  loss_rpn_box_reg: 0.1003 (0.1212)  time: 0.3765  data: 0.0119  max mem: 2775
Epoch: [3]  [1040/1443]  eta: 0:02:31  lr: 0.001000  loss: 0.2732 (0.2808)  loss_classifier: 0.0616 (0.0641)  loss_box_reg: 0.0764 (0.0828)  loss_objectness: 0.0092 (0.0132)  loss_rpn_box_reg: 0.1053 (0.1207)  time: 0.3768  data: 0.0123  max me

Epoch: [4]  [ 720/1443]  eta: 0:04:33  lr: 0.001000  loss: 0.2343 (0.2670)  loss_classifier: 0.0417 (0.0594)  loss_box_reg: 0.0632 (0.0765)  loss_objectness: 0.0079 (0.0121)  loss_rpn_box_reg: 0.0935 (0.1191)  time: 0.3757  data: 0.0117  max mem: 2775
Epoch: [4]  [ 760/1443]  eta: 0:04:18  lr: 0.001000  loss: 0.2157 (0.2654)  loss_classifier: 0.0509 (0.0590)  loss_box_reg: 0.0663 (0.0760)  loss_objectness: 0.0027 (0.0120)  loss_rpn_box_reg: 0.0888 (0.1184)  time: 0.3765  data: 0.0116  max mem: 2775
Epoch: [4]  [ 800/1443]  eta: 0:04:02  lr: 0.001000  loss: 0.2160 (0.2647)  loss_classifier: 0.0447 (0.0589)  loss_box_reg: 0.0738 (0.0762)  loss_objectness: 0.0066 (0.0120)  loss_rpn_box_reg: 0.0896 (0.1176)  time: 0.3749  data: 0.0119  max mem: 2775
Epoch: [4]  [ 840/1443]  eta: 0:03:47  lr: 0.001000  loss: 0.2230 (0.2637)  loss_classifier: 0.0461 (0.0588)  loss_box_reg: 0.0642 (0.0760)  loss_objectness: 0.0075 (0.0119)  loss_rpn_box_reg: 0.1039 (0.1170)  time: 0.3757  data: 0.0118  max me

...

Epoch: [37]  [ 360/1443]  eta: 0:06:51  lr: 0.000010  loss: 0.0898 (0.1296)  loss_classifier: 0.0235 (0.0294)  loss_box_reg: 0.0237 (0.0314)  loss_objectness: 0.0021 (0.0052)  loss_rpn_box_reg: 0.0451 (0.0637)  time: 0.3848  data: 0.0150  max mem: 2775
Epoch: [37]  [ 400/1443]  eta: 0:06:36  lr: 0.000010  loss: 0.1410 (0.1323)  loss_classifier: 0.0392 (0.0301)  loss_box_reg: 0.0346 (0.0319)  loss_objectness: 0.0042 (0.0054)  loss_rpn_box_reg: 0.0783 (0.0648)  time: 0.3807  data: 0.0150  max mem: 2775
Epoch: [37]  [ 440/1443]  eta: 0:06:22  lr: 0.000010  loss: 0.1220 (0.1331)  loss_classifier: 0.0300 (0.0302)  loss_box_reg: 0.0242 (0.0316)  loss_objectness: 0.0037 (0.0055)  loss_rpn_box_reg: 0.0520 (0.0658)  time: 0.3827  data: 0.0157  max mem: 2775
Epoch: [37]  [ 480/1443]  eta: 0:06:06  lr: 0.000010  loss: 0.0944 (0.1321)  loss_classifier: 0.0215 (0.0297)  loss_box_reg: 0.0226 (0.0314)  loss_objectness: 0.0014 (0.0055)  loss_rpn_box_reg: 0.0500 (0.0655)  time: 0.3801  data: 0.0146  ma

Epoch: [38]  [ 160/1443]  eta: 0:08:12  lr: 0.000010  loss: 0.1182 (0.1304)  loss_classifier: 0.0314 (0.0299)  loss_box_reg: 0.0288 (0.0321)  loss_objectness: 0.0028 (0.0054)  loss_rpn_box_reg: 0.0508 (0.0631)  time: 0.3773  data: 0.0133  max mem: 2775
Epoch: [38]  [ 200/1443]  eta: 0:07:55  lr: 0.000010  loss: 0.1216 (0.1296)  loss_classifier: 0.0253 (0.0294)  loss_box_reg: 0.0285 (0.0316)  loss_objectness: 0.0060 (0.0055)  loss_rpn_box_reg: 0.0586 (0.0631)  time: 0.3784  data: 0.0146  max mem: 2775
Epoch: [38]  [ 240/1443]  eta: 0:07:38  lr: 0.000010  loss: 0.1190 (0.1281)  loss_classifier: 0.0241 (0.0292)  loss_box_reg: 0.0251 (0.0311)  loss_objectness: 0.0022 (0.0055)  loss_rpn_box_reg: 0.0601 (0.0624)  time: 0.3759  data: 0.0143  max mem: 2775
Epoch: [38]  [ 280/1443]  eta: 0:07:22  lr: 0.000010  loss: 0.1395 (0.1295)  loss_classifier: 0.0320 (0.0294)  loss_box_reg: 0.0328 (0.0315)  loss_objectness: 0.0071 (0.0056)  loss_rpn_box_reg: 0.0645 (0.0629)  time: 0.3787  data: 0.0145  ma

Epoch: [38]  [1442/1443]  eta: 0:00:00  lr: 0.000010  loss: 0.1446 (0.1284)  loss_classifier: 0.0323 (0.0295)  loss_box_reg: 0.0316 (0.0309)  loss_objectness: 0.0021 (0.0055)  loss_rpn_box_reg: 0.0676 (0.0624)  time: 0.3711  data: 0.0137  max mem: 2775
Epoch: [38] Total time: 0:09:05 (0.3783 s / it)
Epoch: [39]  [   0/1443]  eta: 0:31:34  lr: 0.000010  loss: 0.0626 (0.0626)  loss_classifier: 0.0085 (0.0085)  loss_box_reg: 0.0135 (0.0135)  loss_objectness: 0.0070 (0.0070)  loss_rpn_box_reg: 0.0336 (0.0336)  time: 1.3132  data: 0.7865  max mem: 2775
Epoch: [39]  [  40/1443]  eta: 0:09:21  lr: 0.000010  loss: 0.1278 (0.1264)  loss_classifier: 0.0274 (0.0293)  loss_box_reg: 0.0301 (0.0311)  loss_objectness: 0.0049 (0.0074)  loss_rpn_box_reg: 0.0565 (0.0587)  time: 0.3788  data: 0.0132  max mem: 2775
Epoch: [39]  [  80/1443]  eta: 0:08:50  lr: 0.000010  loss: 0.1032 (0.1217)  loss_classifier: 0.0187 (0.0270)  loss_box_reg: 0.0178 (0.0296)  loss_objectness: 0.0024 (0.0065)  loss_rpn_box_reg:

Epoch: [39]  [1280/1443]  eta: 0:01:01  lr: 0.000010  loss: 0.1448 (0.1294)  loss_classifier: 0.0341 (0.0298)  loss_box_reg: 0.0352 (0.0312)  loss_objectness: 0.0074 (0.0057)  loss_rpn_box_reg: 0.0613 (0.0626)  time: 0.3781  data: 0.0123  max mem: 2775
Epoch: [39]  [1320/1443]  eta: 0:00:46  lr: 0.000010  loss: 0.1039 (0.1292)  loss_classifier: 0.0265 (0.0298)  loss_box_reg: 0.0237 (0.0311)  loss_objectness: 0.0034 (0.0058)  loss_rpn_box_reg: 0.0509 (0.0626)  time: 0.3779  data: 0.0122  max mem: 2775
Epoch: [39]  [1360/1443]  eta: 0:00:31  lr: 0.000010  loss: 0.1151 (0.1292)  loss_classifier: 0.0250 (0.0297)  loss_box_reg: 0.0204 (0.0311)  loss_objectness: 0.0042 (0.0058)  loss_rpn_box_reg: 0.0584 (0.0627)  time: 0.3786  data: 0.0125  max mem: 2775
Epoch: [39]  [1400/1443]  eta: 0:00:16  lr: 0.000010  loss: 0.1137 (0.1289)  loss_classifier: 0.0255 (0.0297)  loss_box_reg: 0.0227 (0.0309)  loss_objectness: 0.0022 (0.0058)  loss_rpn_box_reg: 0.0562 (0.0625)  time: 0.3783  data: 0.0120  ma

Epoch: [40]  [1080/1443]  eta: 0:02:17  lr: 0.000010  loss: 0.1024 (0.1279)  loss_classifier: 0.0231 (0.0295)  loss_box_reg: 0.0309 (0.0306)  loss_objectness: 0.0039 (0.0057)  loss_rpn_box_reg: 0.0490 (0.0621)  time: 0.3804  data: 0.0133  max mem: 2775
Epoch: [40]  [1120/1443]  eta: 0:02:02  lr: 0.000010  loss: 0.1069 (0.1277)  loss_classifier: 0.0280 (0.0295)  loss_box_reg: 0.0252 (0.0306)  loss_objectness: 0.0027 (0.0057)  loss_rpn_box_reg: 0.0494 (0.0619)  time: 0.3776  data: 0.0123  max mem: 2775
Epoch: [40]  [1160/1443]  eta: 0:01:47  lr: 0.000010  loss: 0.0999 (0.1283)  loss_classifier: 0.0254 (0.0295)  loss_box_reg: 0.0286 (0.0308)  loss_objectness: 0.0021 (0.0057)  loss_rpn_box_reg: 0.0453 (0.0623)  time: 0.3792  data: 0.0126  max mem: 2775
Epoch: [40]  [1200/1443]  eta: 0:01:32  lr: 0.000010  loss: 0.1255 (0.1280)  loss_classifier: 0.0261 (0.0295)  loss_box_reg: 0.0289 (0.0307)  loss_objectness: 0.0026 (0.0057)  loss_rpn_box_reg: 0.0538 (0.0621)  time: 0.3801  data: 0.0127  ma